In [1]:
from cellworld import *
import multiprocessing as mp
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import glob
import pickle

import warnings
warnings.filterwarnings("ignore", category=FutureWarning) 

from _src.pose import *
from _src.visibility import *
from _src.itor import *
from _src.multiproc import *

In [2]:
# get logs
logs = glob.glob('./_data/logs/*.json')

# load visibility objects
print('Loading world visibility...')
e = Experiment.load_from_file(logs[3])
vis,w = get_vis(e)

# load visibility matrix
print('Loading visibility matrix...')
[A,V,pts,sparse_arr] = pickle.load(open('./_data/visibility-21_05-1000.pkl','rb'))   
vis_graph = {'V':V,'A':A,'src':pts,'dst':sparse_arr}

# load/build pose library
print('Loading pose library...')
poselib = build_pose_library(logs)

Loading world visibility...
Loading visibility matrix...
Loading pose library...
./_data/logs/pose_library.pkl found, loading...


In [3]:
# make shared dataframe
shared_poselib = SharedPandasDataFrame(poselib)

# make shared arrays
shared_A = SharedNumpyArray(vis_graph['A'])
shared_V = SharedNumpyArray(vis_graph['V'])
shared_src = SharedNumpyArray(vis_graph['src'])
shared_dst = SharedNumpyArray(vis_graph['dst'])

In [ ]:
with mp.Pool(processes=mp.cpu_count()-5) as pool:
    tasks = ((log,shared_poselib,shared_A,shared_V,shared_src,shared_dst) for log in logs)
    result = pool.map(compute_itor_null_fast, tasks)
    for res in tqdm(result, total=len(logs)):
        print(res)
        
shared_poselib.unlink()
shared_A.unlink()
shared_V.unlink()
shared_src.unlink()
shared_ds.unlink()